# Test

In [1]:
from members import Members

test = Members().get_data()

In [2]:
test.head()

,exp_id,memb_id,peak_id,year,season,first_name,last_name,sex,birthdate,yob,...,death,death_date,death_time,death_type,death_height,death_class,summit_bid,summitter,hcn,mchksum
0,TKRG10101,1,TKRG,2010,1,David Alfred,Gottlieb,M,- -,1967.0,...,False,- -,NaN,0,0,0,5,1,0,2437753
1,TKRG10101,2,TKRG,2010,1,Julie,Hentrich,F,- -,1971.0,...,False,- -,NaN,0,0,0,1,17,0,2439311
2,TKRG10101,3,TKRG,2010,1,Joseph N. (Joe),Puryear,M,- -,1973.0,...,False,- -,NaN,0,0,0,5,1,0,2439937
3,AMPG10101,1,AMPG,2010,1,David John,Allen,M,- -,1961.0,...,False,- -,NaN,0,0,0,1,17,0,2435542
4,AMPG10101,2,AMPG,2010,1,David Charles,Barker,M,- -,1964.0,...,False,- -,NaN,0,0,0,1,17,0,2436719


In [3]:
df = Members().clean_data(test)
df.head()

,exp_id,memb_id,peak_id,year,season,sex_M,yob,citizenship,status,residence,...,o2_descent,o2_sleep,o2_medical,death,death_type,death_height,death_class,summit_bid,summit_term,age
0,TKRG10101,1,TKRG,2010,Spring,1.0,1967.0,USA,Leader,"Carlton, Washington",...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,53.0
1,TKRG10101,2,TKRG,2010,Spring,0.0,1971.0,USA,BC Manager,"Carlton, Washington",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,49.0
2,TKRG10101,3,TKRG,2010,Spring,1.0,1973.0,USA,Climber,"Leavenworth, Washington",...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,47.0
3,AMPG10101,1,AMPG,2010,Spring,1.0,1961.0,UK,Leader,"Sheffield, S Yorkshire, England",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,59.0
4,AMPG10101,2,AMPG,2010,Spring,1.0,1964.0,UK,Climber,"Stockport, Derby, England",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,56.0


# Data Sourcing

In [71]:
import os
import pandas as pd

root_dir = os.path.abspath('')
xls_path = os.path.join(root_dir, 'data')
file_names = [f for f in os.listdir(xls_path) if f.endswith('.xls')]

def key_from_file_name(f):
    if f[-4:] == '.xls':
        return f[:-4]

# Create the dictionary
data = {}
for f in file_names:
    data[key_from_file_name(f)] = pd.read_excel(os.path.join(xls_path, f))
    
members = data['members']

In [85]:
data = members.copy()

# EDA

In [74]:
data.shape

(16383, 63)

In [86]:
col_to_drop = ['first_name', 
               'last_name',
               'age',
               'birthdate',
               'route2',
               'route3',
               'ascent2',
               'ascent3',
               'summit_time1', 
               'summit_time2', 
               'summit_time3',
               'summit_date2',
               'summit_date3',
               'o2_note',
               'death_date',
               'death_time',
               'hcn',
               'mchksum']

data.drop(columns= col_to_drop, inplace = True)

# Data Preprocessing

In [87]:
data['season'] = data['season'].map({0 : 'Unknown',
                                             1 : 'Spring',
                                             2 : 'Summer',
                                             3 : 'Autumn',
                                             4 : 'Winter'})

In [88]:
data['death_type'] = data['death_type'].map({0 : 'Unspecified',
                        1 : 'AMS',
                        2 : 'Exhaustion',
                        3 : 'Exposure / frostbite',
                        4 : 'Fall',
                        5 : 'Crevasse',
                        6 : 'Icefall collapse',
                        7 : 'Avalanche',
                        8 : 'Falling rock / ice',
                        9 : 'Disappearance',
                        10 : 'Illness (non-AMS)',
                        11 : 'Other',
                        12 : 'Unknown'})

In [89]:
data['death_class'] = data['death_class'].map({0 : 'Unspecified',
                        1 : 'Death enroute BC',
                        2 : 'Death at BC / ABC',
                        3 : 'Route preparation',
                        4 : 'Ascending in summit bid',
                        5 : 'Descending from summit bid',
                        6 : 'Expedition evacuation',
                        7 : 'Other / Unknown'})

In [90]:
data['summit_bid'] = data['summit_bid'].map({
    0 : 'Unspecified',
    1 : 'No summit bid',
    2 : 'Aborted below high camp',
    3 : 'Aborted at high camp',
    4 : 'Aborted above high camp',
    5 : 'Successful summit bid'})

In [91]:
data.rename(columns={'summitter' : 'summit_term'}, inplace=True)

In [92]:
data['summit_term'] = data['summit_term'].map({
0 : 'Unspecified',
1 : 'Success',
2 : 'Success subpeak',
3 : 'Bad weather',
4 : 'Bad conditions',
5 : 'Accident',
6 : 'AMS',
7 : 'Exhaustion',
8 : 'Frostbite',
9 : 'Illnesses',
10 : 'Logisitic',
11 : 'O2 system failure',
12 : 'Route difficulty',
13 : 'Too late/slow',
14 : 'Assisting',
15 : 'Route/camp/rope preparation',
16 : 'No time left',
17 : 'No climbing',
18 : 'Other',
19 : 'Unknown'})

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

data['summit_date1'] = pd.to_datetime(data.summit_date1, errors = 'coerce')

# le = LabelEncoder()
# data['sex'] = le.fit_transform(data['sex'])

# Combine "age_transformer" and "One hot encoder" into a single preprocessor
preprocessor = ColumnTransformer([
    ('median', SimpleImputer(strategy='median'), ['yob']),
    ('frequency', SimpleImputer(strategy='most_frequent'), ['citizenship', 'status']),
    ('ohe', OneHotEncoder(drop= 'first'), ['sex'])])

# Pass the combined preprocessor into a Pipeline as a single step
final_pipe = Pipeline([('preprocessing', preprocessor)])

data_trans = final_pipe.fit_transform(data)

data[['yob', 'citizenship', 'status', 'sex']] = data_trans

data.rename(columns={'sex' : 'sex_M'}, inplace=True)

data.occupation.fillna('Unknown', inplace = True)
data.residence.fillna('Unknown', inplace = True)

In [103]:
import datetime
now_year = datetime.datetime.now().year
data['age'] = now_year - data['yob']

In [104]:
data

,exp_id,memb_id,peak_id,year,season,sex_M,yob,citizenship,status,residence,...,o2_descent,o2_sleep,o2_medical,death,death_type,death_height,death_class,summit_bid,summit_term,age
0,TKRG10101,1,TKRG,2010,Spring,1.0,1967.0,USA,Leader,"Carlton, Washington",...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,53.0
1,TKRG10101,2,TKRG,2010,Spring,0.0,1971.0,USA,BC Manager,"Carlton, Washington",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,49.0
2,TKRG10101,3,TKRG,2010,Spring,1.0,1973.0,USA,Climber,"Leavenworth, Washington",...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,47.0
3,AMPG10101,1,AMPG,2010,Spring,1.0,1961.0,UK,Leader,"Sheffield, S Yorkshire, England",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,59.0
4,AMPG10101,2,AMPG,2010,Spring,1.0,1964.0,UK,Climber,"Stockport, Derby, England",...,False,False,False,False,Unspecified,0,Unspecified,No summit bid,No climbing,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,AMAD16316,3,AMAD,2016,Autumn,1.0,1954.0,Switzerland,Climber,"Bern, Switzerland",...,False,False,False,False,Unspecified,0,Unspecified,Aborted at high camp,Other,66.0
16379,AMAD16316,2,AMAD,2016,Autumn,0.0,1984.0,Switzerland,Climber,"Bern, Switzerland",...,False,False,False,False,Unspecified,0,Unspecified,Aborted at high camp,Other,36.0
16380,AMAD16316,4,AMAD,2016,Autumn,1.0,1980.0,Switzerland,Climber,"Caux, Vaud, Switzerland",...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,40.0
16381,AMAD16316,8,AMAD,2016,Autumn,0.0,1983.0,Switzerland,Climber,Unknown,...,False,False,False,False,Unspecified,0,Unspecified,Successful summit bid,Success,37.0
